In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
from Bio import SeqIO

def read_fasta_file(file_path, limit=None):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
        if limit is not None and len(sequences) >= limit:
            break
    return sequences

def one_hot_encode(sequence):
      encoding = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'N': [0.25, 0.25, 0.25, 0.25],
        'Y': [0, 0.5, 0, 0.5],
        'R': [0.5, 0, 0.5, 0],
        'M': [0.5, 0.5, 0, 0],
        'K': [0, 0, 0.5, 0.5],
        'S': [0, 0.5, 0.5, 0],
        'W': [0.5, 0, 0, 0.5],
        'B': [0, 0.333, 0.333, 0.333],
        'D': [0.333, 0, 0.333, 0.333],
        'H': [0.333, 0.333, 0, 0.333],
        'V': [0.333, 0.333, 0.333, 0]
    }

     #return np.array([encoding.get(base.upper(), None) for base in sequence if encoding.get(base.upper(), None) is not None])
      return np.array([encoding.get(base.upper(), None) for base in sequence if encoding.get(base.upper(), None) is not None], dtype=np.float32)

def preprocess_fasta_enhancer_sequences(file_path, limit=None):
    sequences = read_fasta_file(file_path, limit=limit)
    encoded_sequences = [one_hot_encode(seq) for seq in sequences]
    return np.array(encoded_sequences)

def extract_sequences(input_file, output_file, label, limit=10000):
    with open(output_file, 'w') as outfile:
        count = 0
        for record in SeqIO.parse(input_file, "fasta"):
            if count >= limit:
                break
            record.id = f"{label}_{count}"
            record.description = f"{label} sequence {count}"
            SeqIO.write(record, outfile, "fasta")
            count += 1

experimentally_derived_file = "your experimental enhancer fasta file here"
transposon_derived_file = "your human transposon fasta file here"

extract_sequences(experimentally_derived_file, extracted_sequences_file, label="experimentally_derived")

X_experimentally_derived = preprocess_fasta_enhancer_sequences(extracted_sequences_file)
X_transposon_derived = preprocess_fasta_enhancer_sequences(transposon_derived_file)

X = np.concatenate((X_experimentally_derived, X_transposon_derived))
y = np.concatenate((np.ones(len(X_experimentally_derived)), np.zeros(len(X_transposon_derived))))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

def data_generator(X, y, batch_size):
    num_samples = len(X)
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = pad_sequences(X[start:end], padding='post')
            y_batch = y[start:end]
            yield X_batch, y_batch

model = Sequential([
    Masking(mask_value=0., input_shape=(None, 4)),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 32
train_generator = data_generator(X_train, y_train, batch_size)
val_generator = data_generator(X_val, y_val, batch_size)

steps_per_epoch_train = len(X_train) // batch_size
steps_per_epoch_val = len(X_val) // batch_size

history = model.fit(train_generator, epochs=50, steps_per_epoch=steps_per_epoch_train, validation_data=val_generator, validation_steps=steps_per_epoch_val)

model.save("model.h5")

import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from Bio import SeqIO

# Load the saved model
model = load_model("model.h5")

# Read and one-hot encode unknown transposon sequences
unknown_transposon_file = "unknown transposons of the same sub class here"
X_unknown_transposon = preprocess_fasta_enhancer_sequences(unknown_transposon_file)
sequences = read_fasta_file(unknown_transposon_file)

# Pad the sequences
X_unknown_transposon_padded = pad_sequences(X_unknown_transposon, padding='post')

# Make predictions
predictions = model.predict(X_unknown_transposon_padded)

# Do something with the predictions, e.g., print them, save them to a file, etc.
for i, pred in enumerate(predictions):
    print(f"Sequence {i}: Probability = {pred[0]}")
with open("predictions.txt", "w") as f:
    for i, pred in enumerate(predictions):
        f.write(f"Sequence {i}: Probability = {pred[0]}\n")
threshold = 0.5
for i, pred in enumerate(predictions):
    if pred[0] > threshold:
        print(f"Sequence {i} is classified as an enhancer with probability {pred[0]}")
    else:
        print(f"Sequence {i} is classified as a non-enhancer with probability {pred[0]}")
# Assume 'predictions' is a list of probabilities obtained from the model
sequence_probabilities = [(i, prob) for i, prob in enumerate(predictions)]

# Sort by probability (descending order for strong enhancers)
sorted_sequences = sorted(sequence_probabilities, key=lambda x: x[1], reverse=True)

# Select the top N sequences as strong enhancers (change N to your desired threshold)
N = 10  # For example, choose the top 10 sequences
strong_enhancers = sorted_sequences[:N]

# Sort by probability (ascending order for weak enhancers)
sorted_sequences = sorted(sequence_probabilities, key=lambda x: x[1])

# Select the top N sequences as weak enhancers (change N to your desired threshold)
N = 10  # For example, choose the top 10 sequences
weak_enhancers = sorted_sequences[:N]

# Print the strong and weak enhancers
print("Strong enhancers:", strong_enhancers)
print("Weak enhancers:", weak_enhancers)

strong_enhancers = []
weak_enhancers = []
mimicking_enhancers = []

for i, prob in enumerate(predictions):
    if prob >= 0.9:
        strong_enhancers.append((i, prob))
    elif 0.5 <= prob < 0.9:
        weak_enhancers.append((i, prob))
    else:
        mimicking_enhancers.append((i, prob))

print("Strong enhancers:", strong_enhancers)
print("Weak enhancers:", weak_enhancers)
print("Mimicking enhancer functions:", mimicking_enhancers)
